# Metodologia: Introdução
A secção de metodologia é dedicada ao detalhamento das diretrizes e princípios que definem o desenvolvimento dos sistemas do dispositivo de reciclagem de garrafas PET em filamento para impressão 3D. De maneira geral, pode-se dividir o funcionamento do dispositivo em 2 grupos principais: elétrica/eletrônica e mecânica, onde a elétrica/eletrônica é responsável pelo controle do sistema e a mecânica faz a integração com atuadores comandados pelo primeiro grupo. O diagrama funcional da eletrônica do dispositivo pode ser descrito como na figura a seguir:

<center><img src="IMG\REC-3DP_DIAGRAMA_FUNCIONAL_COLOR.png"/></center> 

Como esse artigo propõe reaproveitar uma placa Creality3D já desenvolvida, parte da seleção dos periféricos é limitada pela construção de hardware da placa. No que tange à parte de controle, propõe-se elaborar uma interface de comando via display e encoder. Além disso, como a placa também possui interface de comunicação serial via USB, também é possível desenvolver uma interface de comando para controle do dispositivo de maneira totalmente independente, através de um computador.


# Metodologia: Motor
*&emsp; Na placa Creality3D, há 4 drivers para controle dos motores de passo originalmente instalados na impressora 3D. Nesse artigo, apenas um motor será utilizado. O driver instalado nessa placa é o A4988, um controlador de motor de passo de baixo custo e fácil utilização, capaz de fornecer corrente e controle de direção para motores de passo bipolares. Sendo assim, a escolha do motor é limitada a um motor de passo bipolar com 4 fios. O motor escolhido foi o Nema 17 modelo VZS1740-048-1004, fabricado pela Vurtz Motors. Este é um motor de passo bipolar que atende as tensões de operação da placa, com um torque de 4,8Kgf.cm e corrente de operação de até 1,0A. Além disso, possui um ângulo de passo de 1,8°/passo (±5%), ou seja, 200 passos/revolução.

*&emsp; Para o controle de velocidade do motor de passo, foi escolhido o Timer/Counter2 do microcontrolador ATmega1284P, um periférico com resolução de 8 bits que possui alguns recursos como, gerador de forma de onda, gerador de frequência, clock para fator de pré-escala de 10 bits, entre outros. O gerador de forma de onda (WGM) possui alguns modos: Normal, CTC e Fast PWM. Para um simples controle de velocidade de um motor de passo através da modulação da frequência dos pulsos para o driver A4988, diversas estratégias de controle de frequência podem ser adotadas. Para este projeto, a geração de forma de onda no modo CTC (Clear Timer on Compare Match) foi adotado.

*&emsp; O modo CTC utiliza o registrador OCR2A para controlar a resolução do contador. Nesse modo, o contador é reiniciado quando o valor do contador (TCNT2) é igual ao OCR2A. O valor definido no registrador OCR2A determina o limite máximo do contador, influenciando assim a sua resolução. Esse modo possibilita um maior controle sobre a frequência de saída de correspondência de comparação (evento de compare match). A figura a seguir ilustra o diagrama de tempos para o Timer no modo CTC. 
<center><img src="IMG\CTC-Mode-Diagram-Pequeno.png"/></center> 
  
*&emsp; Para produzir uma forma de onda no modo CTC, é possível configurar a saída OC2A para alternar seu nível lógico a cada evento de compare match, selecionando os bits do modo de saída de comparação (COM) como modo de alternância (COM2A1:0 = 1). O valor de OC2A não será visível no pino de saída do microcontrolador até que os dados sejam direcionados para a saída. A frequência máxima da forma de onda gerada será $f_{OC2A_{máx}} = \frac{f_{clk_{I/O}}}{2}$ quando OCR2A estiver configurado como zero (0x00) e o fator de pré-escala for igual a 0x01. A frequência da forma de onda é determinada pela seguinte equação:
$$ f_{OC2A} = \frac{f_{clk_{I/O}}}{2 \cdot N \cdot (1+OCR2A)}$$
onde:  
$f_{clk_{I/O}}$ = frequência de clock do microcontrolador [Hz];  
N = fator de pré-escala (1, 8, 32, 64, 128, 256 ou 1024).  

*&emsp; Essa frequência, no entanto, não é a frequência do sinal que controla os passos (steps) do driver do motor de passo, mas sim a frequência com que o sinal no pino de saída OC2A é alternado. Ou seja, um ciclo completo para um pulso do driver do motor de passo deve contemplar 2 ciclos de alternância de sinal do registrador. Assim, como o período e frequência são inversamente proporcionais, é possível deduzir a fórmula:
$$f_{step} = \frac{f_{OC2A}}{2} = \frac{f_{clk_{I/O}}}{4 \cdot N \cdot (1+OCR2A)}$$

*&emsp; Como a placa Creality3D utilizada possui a finalidade primária de impressão 3D, o driver A4988 é montado na placa com uma configuração de microstepping de 1:16. Microstepping é uma técnica usada para obter um movimento mais suave em motores de passo, dividindo cada etapa completa em etapas intermediárias menores. O que signifca dividir cada passo completo em 16 micropassos. Em um motor de passo, um passo completo representa uma variação angular $\alpha$ calculada por $\alpha = \frac{360°}{n}$, onde $n$ é o número de passos necessários para uma volta completa ou um movimento de 360 graus. Portanto, para o motor utilizado neste projeto, que possui 200 passos por revolução, uma revolução com o uso de microstepping resultará em $200 \cdot 16 = 3200$ passos. A resolução original do motor era de $\alpha = \frac{360°}{200 passos} = 1,8°/passo$, mas com o microstepping, a resolução passa a ser $\alpha = \frac{360°}{3200 passos} = 0,1125°/passo$. É importante destacar que à medida que o fator de microstepping aumenta, o torque do motor diminui. No entanto, para resolver o problema de redução de torque, foram projetadas e fabricadas engrenagens impressas em 3D, que são conectadas ao motor através de um sistema de redução. Mais detalhes sobre esse sistema de redução por associação de engrenagens são fornecidos em outra seção da metodologia.
*&emsp; Tendo definido as equações de frequência de pulsos (steps) e quantidade $n$ de passos do motor (3200), pode-se definir a velocidade efetiva do motor. Comumente, a velocidade de motores é dada em RPM (rotações por minuto), uma unidade de frequência:
$$v_{motor} = f_{motor_{Hz}} \cdot 60 = \frac{1}{T \cdot n} \cdot 60 = \frac{1}{\frac{1}{f_{step}} \cdot n} \cdot 60 = \frac{f_{step}}{n} \cdot 60 = \frac{f_{clk_{I/O}} \cdot 60}{4 \cdot N \cdot (1 + OCR2A) \cdot n}$$
onde:  
$f_{motor_{Hz}}$ = frequência do motor para uma volta com $n$ pulsos [Hz];  
T = período do pulso de um passo [s];  
60 = fator de conversão para $f_{motor_{Hz}}$ de Hz para RPM.  

*&emsp; Com base nos princípios de controle de velocidade do motor de passo para esse dispositivo, é possível implementar esse controle por meio da configuração do timer selecionado e da manipulação do registrador OCR2A, responsável por modular a saída pelo pino OC2A, de acordo com a configuração escolhida. Para esse projeto específico, foi constatado que um fator de pré-escala de 64 abrange uma faixa de velocidades adequada para a operação do dispositivo. Além disso, é essencial desenvolver uma interface que permita selecionar a velocidade desejada, definir o sentido de rotação, bem como o momento de iniciar e interromper o processo de tracionamento. O controle do motor de passo pode ser resumido com o diagrama a seguir:

<center><img src="IMG\REC-3DP_FLUXOGRAMA-MOTOR.png"/></center>

# Metodologia: Hotend
*&emsp; Assim como o controle de velocidade, parte da escolha de hardware do sistema de controle de tmeperatura é limitada pela construção da placa, que já possui entradas preparadas com circuitos com filtro para entrada analógica dos dados do sensor de temperatura e MOSFETs de potência para controle do cartucho aquecedor. Pelo fato do cartucho aquecedor ser um elemento resistivo, o princípio de seu funcionamento é baseado na modulação da tensão aplicada em seus terminais e, consequentemente, a corrente que passa por ele. Ao modular o sinal de gate do MOSFET de potência por largura de pulso (PWM), é possível obter esse resultado. Como a programação é feita pela IDE de Arduino, é válido aproveitar a biblioteca nativa da placa de desenvolvimento com algumas funções que podem ser úteis para essa aplicação. A analogWrite(), por exmeplo, modula um sinal PWM em uma determinada porta a um frequência constante de aproximadamente 490 Hz. Ao controlar um MOSFET de potência por PWM, é necessário atentar-se à frequência do pulso gerado pelo microcontrolador. Frequências muito altas podem causar perdas de comutação, interferência eletromagnética e limitações de resposta do sistema. Frequências muito baixas, por sua vez, podem causar uma resposta lenta e controle impreciso. A frequência de 490 Hz permite uma resposta rápida do sistema de controle, garantindo que o aquecimento seja regulado com precisão. Além disso, essa frequência não é alta o suficiente para sofrer com interferência eletromagnética causada pelas altas taxas de variação de corrente e tensão.

*&emsp; Assim como é feito no próprio controle de temperatura da impressora 3D controlada pela Creality3D, esse artigo propõe o desenvolvimento de um sistema de controle de temperatura em malha fechada com controlador PID. A figura a seguir ilustra seu princípio de funcionamento:

<center><img src="IMG\REC-3DP_DIAGRAMA_CONTROLE.png"/></center>

*&emsp; Nesse sistema, é inserido um sinal de entrada do tipo degrau, denominado set point (SP) e este sinal é comparado com o valor de temperatura atual pelo controlador. O valor de temperatura atual é fornecido pelo sensor de temperatura. O controlador é responsável por comparar o sinal de referência e o valor atual e calcular o erro. A partir do erro, o sinal de saída é calculado a partir das equações e parâmetros PID pré-estabelecidos. Esse sinal de saída analógico é traduzido na forma onda quadrada com largura de pulso modulada de acordo com o cálculo realizado no módulo PID. O MOSFET de potência recebe o sinal e atua no fornecimento de energia para o cartucho aquecedor, de acordo com o sinal recebido. Assim, o com a modulação de tensão pelo cartucho aquecedor e, consequentemente, corrente por ele, o elemento resistivo varia temperatura. Essa mudança de temperatura é sentido pelo sensor de temperatura e realimenta o sistema com um sinal elétrico de volta para o controlador repetir o cálculo do erro e assim ssucessivamente.

*&emsp; Nesse contexo, a fim de definir a estratégia para implementação do sistema de controle PID, é válido explicar alguns princípios norteadores dessa técnica:

1.&emsp; O termo P é proporcional ao erro atual ($SP - PV$) multiplicado por um fator de ganho Kp. Um erro maior resultará em uma saída de controle maior. Usar apenas o controle proporcional levará a um erro estável entre o ponto de ajuste e o valor do processo.

2.&emsp; O termo I integra os valores passados do erro ao longo do tempo. Ele busca eliminar o erro residual acumulado e adiciona um efeito de controle proporcional ao valor cumulativo do erro. À medida que o erro diminui, o efeito proporcional diminui, mas é compensado pelo efeito crescente do termo integral.

3.&emsp; O termo D estima a tendência futura do erro com base na taxa atual de mudança. É conhecido como "controle antecipatório" porque busca reduzir o efeito do erro SP - PV exercendo uma influência de controle com base na velocidade da mudança do erro. Quanto mais rápida a mudança, maior o efeito de controle ou amortecimento.

*&emsp; Tendo explicado os princípios básicos de um controlador PID, é possível prosseguir para a implementação efetiva do sistema. Um fator importante para esse tipo de controle é a taxa de amostragem, tanto da leitura de temperatura, quanto da taxa com que o erro é calculado. A princípio, a fim de economizar recursos do microcontrolador, foi decidido utilizar apenas um timer para determinar a taxa de amostragem e a taxa de cálculo do erro e consequentemente da saída PID em si. É possível configurar um timer para gerar um evento de overflow a uma taxa de tempo conhecida e utilizar rotinas de serviço de interrupção (ISRs) para garantir que a taxa de amostragem e a taxa de cálculo do PID seja constante.

*&emsp; O timer disponível escolhido para lidar com o módulo de temperatura foi o Timer/Counter3, um periférico do microcontrolador de 16 bits com diversos recursos integrados, incluindo controle por PWM e quatro fontes de interrupção independentes. Para controle da taxa de amostragem e atualização dos parâmetros PID, foi escolhido utilizar um timer para gerar eventos de interrupção a cada 0,5 segundos (2 Hz). Com o timer configurado, acessamos o evento de interrupção através da ISR correspondente. O controle da frequência de atualização pode ser descrita como na equação a seguir:

$$f_{timer} = \frac{f_{clk_{I/O}}}{N \cdot (2^n - TCNT3)}$$

onde:  
$f_{timer}$ = frequência desejada [Hz];  
$f_{clk_{I/O}}$ = frequência de clock do microcontrolador [Hz];  
$N$ = fator de pré-escala;  
$n$ = número de bits do registrador;  
$TCNT3$ = pré-carga no registrador que determinará o tempo para ocorrer o evento de overflow.  


# Resultados: Motor
*&emsp; Para fins de teste e validação, foi adotado um valor arbitrário de $OCR2A = 16$. Adotando um clock de CPU de 16 MHz e fator de pré-escala de 64, podemos calcular a velocidade do motor:
$$v_{motor} = \frac{16000000 \cdot 60 }{4 \cdot 64 \cdot (1+16) \cdot 3200} = 68,93 RPM$$

*&emsp; Em laboratório, a fim de testar a fidelidade da velocidade calculada e a velocidade real do motor, foi utilizado um tacômetro Minipa MDT-2245C (precisão de $\pm$ 0,05%) e a velocidade medida para essa mesma configuração foi de 69 RPM. Portanto, pode-se concluir que foi estabelecido um controle de velocidade satisfatório para o sistema, ainda que essa não seja a variável de processo principal a ser controlada.

# Resultados: Hotend
*&emsp; Tendo definido e implementado o controlador PID em programação de texto estruturado C/C++ e gravado o programa no microcontrolador, foi possível dar início aos testes de controle de temperatura. Alguns firmwares de impressora 3D de código aberto utilizam uma temperatura de referência média para calibrar os parâmetros proporcional ($K_{p}$), integral ($K_{i}$) e derivativo ($K_{d}$), como é o caso do Marlin, um driver para impressoras RepRap de código aberto. Como o elemento de aquecimento escolhido para o controle de temperatura é um produto já comercializado em impressoras 3D, optou-se por sintonizar os parâmetros PID com a mesma temperatura de referência utilizada no firmware Marlin: 150°C. Na figura a seguir, é possível observar os resultados obtidos de resposta ao degrau de entrada com diferentes valores de ganho proporcionais, ainda sem termos integral e derivativo:

<center><img src="IMG\Testes_SP-150-Pequeno.png"/></center> 

No teste do gráfico anterior, foi possível observar a resposta ao degrau com ganho proporcional variando de 1 até 35. Nota-se que, para o setpoint (SP) escolhido, ganhos proporcionais < $K_{p} = 9$ o sistema sequer atinge a temperatura desejada. Somente a partir de $K_{p} = 9$ a temperatura desejada é atingida, ainda que somente em seu valor de pico, estabilizando abaixo do SP. Além disso, nota-se também a presença de uma curva característica para controle de sistemas: o ganho proporcional crítico, que inicia uma oscilação constante em um sistema de controle. Esse comportamento é observado quando $K_{p} = 35$, conforme a figura a seguir:

<center><img src="IMG\Testes_SP-150_Kp-35_Limpo-Pequeno.png"/></center> 

Esse ganho, de acordo com o método do período crítico de Ziegler-Nichols, pode ser usado para sintonizar os parâmetros proporcional, integral e derivativo em controladores PID. O método se resume em duas etapas principais:
* Com termos integral ($K_{i}$) e derivativo ($K_{d}$) do controlador zerados, aumentar o ganho porporcional $K_{p}$ até que a resposta do sistema oscile periodicamente. Esse ganho é denominado ganho proporcional crítico ($K_{u}$);
* Medir o período da oscilação ao encontrar $K_{u}$. O período da oscilação é comumente representado por $P_{u}$.  

Tendo determinado os termos $K_{u}$ e $P_{u}$, pode-se calcular os parâmetros PID com as seguintes equações:

$$K_{p} = 0,6 \cdot K_{u}$$
$$K_{i} = \frac{2}{P_{u}}$$
$$K_{d} = \frac{P_{u}}{8}$$

Além da sintonia de um controlador PID, o método de Ziegler-Nichols também contempla outros contorladores, como PI e PD. As seguintes equações valem para um controlador PI:

$$K_{p} = 0,45 \cdot K_{u}$$
$$K_{i} = \frac{1,2}{P_{u}}$$

Sabendo que o ganho proporcional crítico gerou um período de oscilação de 30s, foram calculados os parâmetros para um controlador PI, conforme o método de Ziegler-Nichols. A figura a seguir ilustra a resposta do sistema:

<center><img src="IMG\Testes_SP-150_Kp-1575_Ki-004_Pequeno.png"/></center> 

Observa-se que o sistema tende a estabilizar próximo de 151,09°C, o que resulta em um erro de estado estacionário ($e_{ss}$) = 1,09°C, cerca de 0,73% do SP. Esse valor, no entanto, está longe de ser ótimo, pois ele só é estabiliza cerca de 9 minutos depois do início do processo. Sabe-se que impressoras 3D atingem temperaturas até mais altas para fusão do material e estabilizam a temperatura em cerca de 2 minutos. Ademais, nota-se um sobressinal considerável no início do aquecimento: o sistema atinge um valor de pico de 170,94°C, 13,96% do SP. Esse sobressinal, apesar de não trazer grandes riscos para a situação de sintonia onde o SP é de 150°C, pode apresentar risco de danificar o sistema ou até de incêndio em situações normais de trabalho. Espera-se realizar o trabalho de reciclagem do filete de garrafa PET em filamento numa faixa de trabalho de 220 a 230°C. Se aplicarmos um sobressinal de 13,96% a um SP de 230°C, pode-se atingir um pico de 262,11°C, teoricamente. Essa temperatura, no entanto, já está fora da temperatura de trabalho permitida em impressoras como a Creality Ender 3, por exemplo. Uma alternativa para tratar do problema de sobressinal seria diminuir o ganho $K_{i}$. Outra alternativa, no entanto, é a inserção do termo derivativo $K_{d}$ no sistema, que possui a capacidade de reduzir o sobressinal, além de reduzir o tempo de resposta do sistema. A figura a seguir ilustra a resposta da implementação de um controlador PID, de acordo com o método de Ziegler-Nichols:

<center><img src="IMG\Testes_SP-150_Kp-21_Ki-0067_Kd-375-Pequeno.png"/></center> 

Ao recalcular os parâmetros PID de acordo com as equações de Ziegler-Nichols, foi obtido um $K_{i}$ maior. Assim, apesar da presença do termo derivativo $K_{d}$ no sistema, ainda temos a presença de um sobressinal significativo no sistema de 17,55%. Outro ponto a se observar é que o sistema estabilizou ligeiramente mais rápido por volta de 150,87°C, representando um $e_{ss}$ de 0,87°C, 0,58% do SP. Ainda assim, os resultados obtidos não são ideais para a aplicação proposta. O grupo propôs-se, então, a realizar a sintonia pelo método de tentativa e erro, aumentando gradativamente somente o ganho proporcional $K_{p}$ e posteriormente os ganhos integral e derivativo $K_{i}$ e $K_{d}$, respectivamente, de modo a otimizar o sobressinal, o erro estacionário e o o tempo de acomodação do sistema. A figura a seguir ilustra os parâmetros encontrados e a resposta do sistema aos novos parâmetros PID:

<center><img src="IMG\Testes_SP-150_Kp-18_Ki-0006_Kd-160-Pequeno.png"/></center> 

Com os parâmetros encontrados pelo método de tentativa erro, foi obtido um $e_{ss}$ de 0,2°C, 0,13% do SP. Além disso, observa-se também a ausência de sobressinal e um rápido tempo de acomodação, em cerca de 83 segundos. O sistema possui um perfil bem conservador com um ganho derivativo consideravelmente alto, de modo a evitar o sobressinal. Além disso, o ganho derivativo mais alto atribui maior sensibilidade às variações de temperatura do sistema. Apesar dessa característica não ser muito desejável em sistemas de controle de temperatura em função do tempo de resposta mais lento em comparação com outros sistemas que integram controladores PID, pode-se verificar que o efeito produzido é satisfatório. Abaixo temos uma comparação da resposta do sistema com controladores PI (Ziegler-Nichols), PID (Ziegler-Nichols) e PID otimizado:

<center><img src="IMG\Testes_SP-150_Comparacao-Pequeno.png"/></center>

Apesar do método de Ziegler-Nichols ter uma taxa de eficiência relativamente alta para sintonia de controladores PID, em alguns casos - como nesse sistema - pode não ser o método mais adequado. Uma explicação provável para a baixa eficiência do método de Zigler-Nichols nesse caso é a presença de um tempo morto de cerca de 5 segundos no sistema. Tempo morto é o atraso ou a falta de resposta do sistema a uma mudança no estímulo de entrada (degrau de 150°C). O método do período crítico de Ziegler-Nichols não contempla a presença do tempo morto e isso pode influenciar na resposta do sistema. Literaturas como [encontrar um livro que cite métodos alternativos] citam outros métodos para tratar desse problema, como Cohen-Coon, Kappa-Tau, entre outros*.  

*https://www.incatools.com/pid-tuning/pid-tuning-methods/#:~:text=In%20broad%20terms%2C%20there%20are,%2C%20and%20model%2Dbased%20tuning.

Como o método de tentativa e erro apresentou um comportamento satisfatório, optou-se por permanecer com ele. A fim de validar a eficiência do controle de temperatura para situações de trabalho, foi adotado um SP de 220°C, conforme demonstrado por TYLMAN e DZIERZEK (2020). A figura a seguir demonstra a resposta do sistema com PID otimizado por tentativa e erro e degrau de entrada de 220°C:

<center><img src="IMG\Testes_SP-220_Kp-18_Ki-0006_Kd-160-Pequeno.png"/></center>

Em temperaturas de trabalho ideais, observou-se um sistema relativamente estável com mais ruído no resposta - possivelmente em função da alta sensibilidade gerada pelo ganho derivativo mais alto. Esse ruído, no entanto, não chega a ser significativo, haja visto que o erro não chega a ser maior do que 1,43°C, 0,65% do SP. Ademais, a fim de verificar a estabilidade do sistema a perturbações externas (como a trefilação/extrusão do filete de garrafa PET, por exemplo), foi feito um teste no qual foi passado um filete de garrafa PET pela matriz aquecida. Nesse teste, não foram observadas grandes variações de temperatura além do que fora registrado na figura anterior.

No que tange ao teste do filamento produzido, foi obtido um filamento com superfície uniforme e diâmetro de (1,75 ± 0,025)mm. A figura a seguir apresenta uma amostra produzida.

<center><img src="IMG\Resultado_Filamento_Comp-Pequeno.png"/></center>

Ainda sobre o filamento produzido, uma análise em corte de uma secção transversal evidencia que o material não preenche a área por completo. Com auxílio da ferramenta de CAD Fusion 360, da Autodesk, é possível estimar a área coberta por material:

<center><img src="IMG\Resultado_Filamento2.png"/></center>

A secção transversal do filamento produzido ocupa uma área de aproximadamente 2,177mm², cerca de 90,51% da área total para um filamento sólido com diâmetro de 1,75mm.
